In [30]:
import os
from Bio import SeqIO
from tqdm import tqdm
from pathlib import Path

In [5]:
!export PATH=$PATH:/home/brett/work/OrthogonalTrainValSplits/hashFrag/src

In [1]:
%%bash

hashFrag

usage: hashFrag [-h]
                {blastn,filter_false_positives,identify_homologous_groups,create_orthogonal_splits}
                ...

positional arguments:
  {blastn,filter_false_positives,identify_homologous_groups,create_orthogonal_splits}

optional arguments:
  -h, --help            show this help message and exit


# Introduction

This notebook provides a basic tutorial on the core functionality of hashFrag.

Working with a small MPRA dataset of 10,000 sequences (provided in the `data` directory), users can see example calls to the hashFrag. Specifically, this notebook will encompass the following tasks:

1. `hashFrag blastn`: Identifying candidate pairs of sequences exhibiting similarity in the dataset.
2. (Optional) For each candidate pairing, compute their optimal local alignment score ([Smith-Waterman algorithm](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm)).
3. `hashFrag filter_false_positives`: Using the local alignment scores calculated in the previous step, filter false-positive candidates based on a specified threshold.
    + Note that if the previous step was NOT performed, false-positive candidates will be filtered based on the heuristic local alignment scores provided by the BLAST algorithm.
    + The threshold to use for tuning is not only dataset-dependent, but also highly depends on BLAST (and, if performed, Smith-Waterman) local alignment parameters. See the advanced tutorial (TODO) for an example on our recommended workflow to tune relevant parameters and select an appropriate threshold.
4. `hashFrag identify_homologous_groups`: determine the different subgroups of sequences exhibiting homology.
5. `hashFrag create_orthogonal_splits`: create homology-aware data splits.

hashFrag is a command line tool. This notebook serves as a resource to better understand each step in the process.


# Section 0: Setup

In [3]:
data_dir   = "/home/brett/work/OrthogonalTrainValSplits/hashFrag/data"
fasta_path = os.path.join(data_dir,"K562.sample_10000.fa.gz")
label      = os.path.basename(fasta_path).replace(".fa.gz","")
score_path = os.path.join(data_dir,f"{label}.pairwise_scores.csv.gz")
work_dir   = os.path.join(data_dir,f"{label}.hashFrag.work")
Path(work_dir).mkdir(parents=True,exist_ok=True)
print(work_dir)

blast_dir = os.path.join(work_dir,"blast_partitions")
Path(blast_dir).mkdir(parents=True,exist_ok=True)

/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work


# Section 1 - Identifying candidate similar sequences

The process of identifying candidate pairs of similar sequences involves first creating a BLAST database of the dataset, and then querying each sequence against the database, collecting all pairwise matches that represent potential cases of homology. 

To effectively mitigate potential biases caused by homology-based data leakage, it is imperative that all cases of homology are successfully identified. To that end, we configure the BLASTn call such that recall is maximized (this comes at the expense of increased false-positives, which are subsequently filtered). Here we consider the following parameters of BLASTn:

* word_size: smaller word sizes results in more exact word matches found between the query and sequences in the database, leading to more alignment score calculations being initialized.
* max_target_seqs: set to the size of the database to remove any constraints and allow for all possible candidate sequences to be returned for a given query.
* evalue: the e-value statistic is a measure of how likely you observe the alignment by chance (lower value corresponds to less likely to observe). By increasing the e-value threshold, matches that could be due to chance are returned.
* dust: by setting dust off, low-complexity (e.g., repetitive sequences) are no longer masked/filtered out.

In [6]:
%%bash

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work

WORD_SIZE=7
MAX_TARGET_SEQS=10000 # size of dataset
E_VALUE=100
DUST=no

hashFrag blastn -f $FASTA_PATH -w $WORD_SIZE -e $E_VALUE -d $DUST -o $WORK_DIR



Building a new DB, current time: 12/10/2024 08:00:47
New DB name:   /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.sample_10000.blastdb
New DB title:  K562.sample_10000
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 10000 sequences in 0.260447 seconds.



BLAST DataBase construction finished and written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.sample_10000.blastdb

BLASTn process finished and written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.sample_10000.blastn.out


# Section 3: Filter false-positives based on a defined threshold

The next step involves filtering candidate pairings with alignment scores lower than the specified threshold. There are two different modes of hashFrag depending on what alignment score is selected.

* `hashFrag-lightning` is the faster version where the alignment score computed from the BLAST output file. BLASTn is a heuristic method and the alignment scores were found to highly correlate with the optimal alignment scores; however, its underestimation of homology in some cases can lead to slightly worse recall. 
* `hashFrag-pure` is the slower but more comprehensive method that is based on the optimal, Smith-Waterman local alignment scores between pairs of sequences. The calculation of optimal alignment scores incurs an additional cost to filtering.

An alignment score threshold of 60 was determined to be appropriate based on an analysis looking at alignment scores between dinucleotide shuffled (i.e., random) sequences.

## Section 3.1: Lightning mode

In [17]:
%%bash

INPUT_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.sample_10000.blastn.out.gz
METHOD=lightning

THRESHOLD=60
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work

hashFrag filter_false_positives -m $METHOD -i $INPUT_PATH -t $THRESHOLD -o $WORK_DIR

Filtered results written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag_lightning.similar_pairs.tsv.gz


## Section 3.2: Pure mode (optional)

In [19]:
%%bash

SPLIT_SIZE=100000
BLAST_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.sample_10000.blastn.out
BLAST_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions

LABEL=$( basename -s ".out" $BLAST_PATH )

mkdir -p $BLAST_DIR
cd $BLAST_DIR

split -l $SPLIT_SIZE -a 4 --additional-suffix=.tsv $BLAST_PATH ${LABEL}.partition_

In [21]:
%%bash

cd ../src/external

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
PARTITIONED_BLAST_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions

for PARTITIONED_BLAST_PATH in $PARTITIONED_BLAST_DIR/*.blastn.partition_*.tsv
do
    echo $PARTITIONED_BLAST_PATH
    bash compute_blast_candidate_SW_scores.sh $FASTA_PATH $PARTITIONED_BLAST_PATH
done

/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K562.sample_10000.blastn.partition_aaaa.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K562.sample_10000.blastn.partition_aaab.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K562.sample_10000.blastn.partition_aaac.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K562.sample_10000.blastn.partition_aaad.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K562.sample_10000.blastn.partition_aaae.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K562.sample_10000.blastn.partition_aaaf.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions/K

In [22]:
%%bash

PARTITIONED_BLAST_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/blast_partitions
WORK_DIR=$( dirname $PARTITIONED_BLAST_DIR )

zcat $PARTITIONED_BLAST_DIR/*.pairwise_scores.tsv.gz | gzip > $WORK_DIR/K562.sample_10000.blastn_candidates.custom_scores.tsv.gz

In [23]:
%%bash

INPUT_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.sample_10000.blastn_candidates.custom_scores.tsv.gz
METHOD=pure

THRESHOLD=60
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work

hashFrag filter_false_positives -m $METHOD -i $INPUT_PATH -t $THRESHOLD -o $WORK_DIR

Filtered results written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag_pure.similar_pairs.tsv.gz


# Section 4: Determine groups of homology

There are often distinct groups of sequences exhibiting different cases of homology throughout the dataset. To determine such groups, we represent the "hits" (i.e., pairs of sequences with an alignment score greater than the threshold) as a sparse adjacency matrix. A graph can then be constructed, where nodes correspond to sequences and edges denote shared homology between the two sequences. The process of identifying groups of homology can readily be solved by identifying disconnected subgraphs. 

An efficient implementation for this graph-based task is provided in the `igraph` Python library.

In [25]:
%%bash

HITS_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag_lightning.similar_pairs.tsv.gz
OUTPUT_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/homologous_groups.csv

hashFrag identify_homologous_groups -i $HITS_PATH -o $OUTPUT_PATH

1114 sequences exhibiting homology.
90 homologous groups identified.
Homologous groups written to file: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/homologous_groups.csv


# Section 5: Use cases of hashFrag

Upon identifying groups of sequences exhibiting high similarity (i.e., homology), there are a few ways in which this information can be used for sequence-to-expression model training. These functionalities include: filtering homology spanning existing train-test data splits, creating new, homology-aware (i.e., orthogonal) train-test data splits, or stratifying an existing test split based on homology scores with respect to the train split.

## Section 5.1: Filtering homology from existing splits

In [34]:
import gzip
import random

In [40]:
fasta_path = "/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz"
work_dir = "/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work"
train_split_path = os.path.join(work_dir,"K562.train.fa.gz")
test_split_path = os.path.join(work_dir,"K562.test.fa.gz")

In [44]:
random.seed(21)

fasta_dict = {}
with gzip.open(fasta_path,"rt") as handle:
    for record in SeqIO.parse(handle,"fasta"):
        fasta_dict[record.id] = str(record.seq)

ids = list(fasta_dict)

p_train = 0.8
n_train = int(len(ids)*p_train)
train_ids = random.sample(ids,k=n_train)
with gzip.open(train_split_path,"wt") as handle:
    for sample_id in train_ids:
        handle.write(f">{sample_id}\n{fasta_dict[sample_id]}\n")

test_ids = set(ids).difference(set(train_ids))
with gzip.open(test_split_path,"wt") as handle:
    for sample_id in test_ids:
        handle.write(f">{sample_id}\n{fasta_dict[sample_id]}\n")

In [45]:
%%bash

TRAIN_FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.train.fa.gz
TEST_FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.test.fa.gz
HITS_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag_lightning.similar_pairs.tsv.gz

hashFrag filter_existing_splits --train_fasta_path $TRAIN_FASTA_PATH --test_fasta_path $TEST_FASTA_PATH --hits_path $HITS_PATH

199 sequences filtered from test split.
Filtered results written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/K562.test.filtered.fa.gz


## Section 5.2: Creating homology-aware splits

In [27]:
%%bash

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
HOMOLOGY_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/homologous_groups.csv
OUT_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work

hashFrag create_orthogonal_splits -f $FASTA_PATH -i $HOMOLOGY_PATH -n 10 -o $OUT_DIR

Writing splits...
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_001.csv.gz
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_002.csv.gz
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_003.csv.gz
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_004.csv.gz
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_005.csv.gz
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_006.csv.gz
  /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.work/hashFrag.train_8000.test_2000.split_007.csv.gz
  /home/brett/work/Orth

## Section 5.3: Homology-based stratification of test split